In [1]:
import os
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.append(p)

from waveorder.io.writer import WaveorderWriter
import numpy as np
import zarr

### Initialize the writer

This will instantiate the writer class, no zarr stores, subfolders, etc have been created yet.  This step allows you to define which type of writer you want and let it know your directory to save the data

In [2]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop', 'physical')

### Set Position and Create Zarr

'create_zarr_root()' will create an empty zarr store within the save directory.  No arrays have been added here.  Name of the store must be input by the user.

'create_position()' will create a subgroup in your zarr store with the corresponding position number.  The current working directory of the writer is now this subfolder.  If you specify a prefix, it will be added to the group name

In [3]:
writer.create_zarr_root('waveOrder_Writer_Example.zarr')

Creating new zarr store at /Users/cameron.foltz/Desktop/waveOrder_Writer_Example.zarr


In [4]:
for i in range(10):
    writer.create_position(i, prefix=None)

Creating and opening subgroup Pos_000.zarr
Creating and opening subgroup Pos_001.zarr
Creating and opening subgroup Pos_002.zarr
Creating and opening subgroup Pos_003.zarr
Creating and opening subgroup Pos_004.zarr
Creating and opening subgroup Pos_005.zarr
Creating and opening subgroup Pos_006.zarr
Creating and opening subgroup Pos_007.zarr
Creating and opening subgroup Pos_008.zarr
Creating and opening subgroup Pos_009.zarr


### Check which position (subgroup) the writer is currently at

Useful function for when you are writing to different groups at different times.  i.e. 'position first' writing (example later)

In [5]:
print(f'Current store is at: {writer.get_current_group()}')

Current store is at: Pos_009.zarr


### Change to different position

if your group has a prefix, it must be specified

In [6]:
writer.open_position(0, prefix=None)

Opening subgroup Pos_000.zarr


### Create some fake data to save later

In [7]:
P = 10
T = 3
C = 2
Z = 65
Y = 256
X = 256

data = np.random.rand(P, T, C, Z, Y, X)

In [8]:
np.shape(data)

(10, 3, 2, 65, 256, 256)

### initialize the array which sets the data saving parameters

Here is where you will initialize the array in your current writer.store .  data_shape should match the (T, C, Z, Y, X) dimensions of your data.  Positions are separated by zarr subgroups under the root zarr store.

__chunk_size__ determines how zarr will chunk your data.  This means that when you later try to load the data, it will load one chunk at a time with this specified size.  To have the chunk be one z-slice, you would set chunk_size = (1,1,1,Y,X)

__chan_names__ describe the names of the channels of your data in the order in which they will be written.

__clims__ corresponds to the the display contrast limits in the metadata for every channel, if none, default values will be used

In [9]:
data_shape = (3,2,65,256,256)
chunk_size = (1,1,1,256,256)
chan_names = ['DAPI', 'Phase3D']
clims = [(0,1000),(-0.3, 0.3)]
dtype = 'float32'

In [10]:
writer.init_array(data_shape, chunk_size, chan_names, clims, dtype)

### Write the data along specific dimensions

In [11]:
writer.write(data[0], t=[0,3], c=[0,2], z=[0,65])

# Add a new writer for to save to the same store

In [12]:
writer_denoised = WaveorderWriter('/Users/cameron.foltz/Desktop/waveOrder_Writer_Example.zarr', 
                                  'physical', alt_name = 'physical_data_denoised')

writer_stokes = WaveorderWriter('/Users/cameron.foltz/Desktop/waveOrder_Writer_Example.zarr','stokes')

Opening existing store at /Users/cameron.foltz/Desktop/waveOrder_Writer_Example.zarr
Opening existing store at /Users/cameron.foltz/Desktop/waveOrder_Writer_Example.zarr


In [13]:
writer_denoised.open_position(0)
writer_stokes.open_position(0)

Opening subgroup Pos_000.zarr
Opening subgroup Pos_000.zarr


In [14]:
writer_denoised.init_array(data_shape, chunk_size, chan_names, clims, dtype)
writer_denoised.write(data[0], t=[0,3], c=[0,2], z=[0,65])

writer_stokes.init_array(data_shape, chunk_size, chan_names, clims, dtype)
writer_stokes.write(data[0], t=[0,3], c=[0,2], z=[0,65])

# Complex, 'Position First' Acquisition

Let's say you have a dataset with 10 position with each position containing a data of size (T, C, Z, Y, X) = (3, 2, 65, 256, 256) and you want to save the first time point of every position first, and then move on to the next timepoint.  This will show you how to properly loop through this example case.

In [15]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop', 'physical')
writer.create_zarr_root('waveOrder_Writer_Example_MultiPos.zarr')

Creating new zarr store at /Users/cameron.foltz/Desktop/waveOrder_Writer_Example_MultiPos.zarr


In [16]:
# Must first initialize all of the arrays, since you will be returning to them 
# at different times in the data-saving process.
num_pos = 10

for i in range(num_pos):
    writer.create_position(i, prefix=None)
    writer.init_array(data_shape, chunk_size, chan_names, clims, dtype)
    
# Now begin the rime loop 
for t in range(3):
    
    # At each time point, write data for all positions
    for pos in range(num_pos):
        
        print(f'Writing Position {pos} at Time {t}')
        
        #change to the correct position group
        writer.open_position(pos)
        
        #write this position data
        writer.write(data[pos, t], t=t, c=[0,2], z=[0,65])

Creating and opening subgroup Pos_000.zarr
Creating and opening subgroup Pos_001.zarr
Creating and opening subgroup Pos_002.zarr
Creating and opening subgroup Pos_003.zarr
Creating and opening subgroup Pos_004.zarr
Creating and opening subgroup Pos_005.zarr
Creating and opening subgroup Pos_006.zarr
Creating and opening subgroup Pos_007.zarr
Creating and opening subgroup Pos_008.zarr
Creating and opening subgroup Pos_009.zarr
Writing Position 0 at Time 0
Opening subgroup Pos_000.zarr
Writing Position 1 at Time 0
Opening subgroup Pos_001.zarr
Writing Position 2 at Time 0
Opening subgroup Pos_002.zarr
Writing Position 3 at Time 0
Opening subgroup Pos_003.zarr
Writing Position 4 at Time 0
Opening subgroup Pos_004.zarr
Writing Position 5 at Time 0
Opening subgroup Pos_005.zarr
Writing Position 6 at Time 0
Opening subgroup Pos_006.zarr
Writing Position 7 at Time 0
Opening subgroup Pos_007.zarr
Writing Position 8 at Time 0
Opening subgroup Pos_008.zarr
Writing Position 9 at Time 0
Opening su